In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('../data/transactions.xlsx')
df.head()

,UUID,IdCustomer,DtTransaction,Points
0,fb4e3102-6125-414e-b086-e7f641e38f93,5f8fcbe0-6014-43f8-8b83-38cf2f4887b3,2024-01-27 11:53:23.774,1
1,036b8d66-8adf-479b-ba0f-cb0e3207acb0,5f8fcbe0-6014-43f8-8b83-38cf2f4887b3,2024-01-27 11:53:24.643,1
2,10166182-a164-4608-8cdf-35e4450b2bd7,5f8fcbe0-6014-43f8-8b83-38cf2f4887b3,2024-01-27 12:11:44.728,500
3,03974c58-e86e-4b6b-8143-8f89ca870d09,5f8fcbe0-6014-43f8-8b83-38cf2f4887b3,2024-01-27 12:19:41.063,1
4,e84703a6-b606-4096-a2f3-514971c16c5c,5f8fcbe0-6014-43f8-8b83-38cf2f4887b3,2024-01-27 12:31:46.288,500


In [3]:
condicao = df['IdCustomer'] == '5f8fcbe0-6014-43f8-8b83-38cf2f4887b3'
df_user = df[condicao]
df_user['Points'].sum()

4405

In [4]:
df_summary = df.groupby(['IdCustomer'])['Points'].sum()
df_summary.head()

IdCustomer
000ff655-fa9f-4baa-a108-47f581ec52a1     52
0097ab76-4637-4ece-8ebc-ab6abd61d662    152
012bc897-e45c-4e6a-a742-7e00fa2e59e5     51
014491f5-caa3-4122-be69-a31411ad4bd7     53
014b7774-561a-4a58-93d7-070667df90fb    126
Name: Points, dtype: int64

In [5]:
# reseta o indice para criar um DF
df_summary = df_summary.reset_index()

In [6]:
condicao = df_summary['IdCustomer'] == '5f8fcbe0-6014-43f8-8b83-38cf2f4887b3'
df_summary[condicao]

,IdCustomer,Points
293,5f8fcbe0-6014-43f8-8b83-38cf2f4887b3,4405


In [7]:
# montando um RFV (Recência, Frequência e Valor)
df_RFV = df.groupby(['IdCustomer'])\
    .agg(
        {'Points' : 'sum',
         'UUID' : 'count',
         'DtTransaction' : 'max'
         })\
    .rename(columns={
        'Points' : "Pontos",\
        'UUID' : "Frequencia",
        'DtTransaction' : 'UltimaData'
        }).reset_index()
df_RFV

,IdCustomer,Pontos,Frequencia,UltimaData
0,000ff655-fa9f-4baa-a108-47f581ec52a1,52,3,2024-03-04 12:00:53.979
1,0097ab76-4637-4ece-8ebc-ab6abd61d662,152,5,2024-02-08 13:56:46.749
2,012bc897-e45c-4e6a-a742-7e00fa2e59e5,51,2,2024-02-05 12:24:09.715
3,014491f5-caa3-4122-be69-a31411ad4bd7,53,4,2024-02-29 22:28:54.356
4,014b7774-561a-4a58-93d7-070667df90fb,126,28,2024-02-06 13:22:58.636
...,...,...,...,...
722,fbd9b6fd-0b1a-436c-822e-7c4dd4fe0c70,50,1,2024-02-05 13:58:26.275
723,fcbb3e3a-e147-4a13-9f24-80e33f3ae194,58,9,2024-02-06 14:05:39.814
724,fe348fe6-a2f7-4f3f-b932-c10ad0b5692e,50,1,2024-02-23 11:58:58.401
725,fe59fd95-d131-4d50-bb6e-10cdfa7d5f47,328,83,2024-03-01 12:03:12.124


In [8]:
import datetime

data1 = df['DtTransaction'][0]
print(data1)
print(type(data1))

now = datetime.datetime.now()
print((now - data1))
print((now - data1).days)

2024-01-27 11:53:23.774000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
66 days 01:10:47.700663
66


In [9]:
def recencia(x):
    diff = datetime.datetime.now() - x.max()
    return diff.days

In [10]:
# montando um RFV (Recência, Frequência e Valor)
df_RFV = df.groupby(['IdCustomer'])\
    .agg(
        {'Points' : 'sum',
         'UUID' : 'count',
         #'DtTransaction': ['max', recencia] #mostrar data e nº dias
         'DtTransaction': recencia
         })\
    .rename(columns={
        'Points' : "Pontos",\
        'UUID' : "Frequencia",
        'DtTransaction' : 'UltimaData'
        }).reset_index()
df_RFV

,IdCustomer,Pontos,Frequencia,UltimaData
0,000ff655-fa9f-4baa-a108-47f581ec52a1,52,3,29
1,0097ab76-4637-4ece-8ebc-ab6abd61d662,152,5,53
2,012bc897-e45c-4e6a-a742-7e00fa2e59e5,51,2,57
3,014491f5-caa3-4122-be69-a31411ad4bd7,53,4,32
4,014b7774-561a-4a58-93d7-070667df90fb,126,28,55
...,...,...,...,...
722,fbd9b6fd-0b1a-436c-822e-7c4dd4fe0c70,50,1,56
723,fcbb3e3a-e147-4a13-9f24-80e33f3ae194,58,9,55
724,fe348fe6-a2f7-4f3f-b932-c10ad0b5692e,50,1,39
725,fe59fd95-d131-4d50-bb6e-10cdfa7d5f47,328,83,32
